# Model comparisons

In [1]:
from face_alignment import FaceAlignment, LandmarksType

from loreal_poc.dataloaders.loaders import DataLoaderFFHQ
from loreal_poc.dataloaders.wrappers import (
    CroppedDataLoader,
    ResizedDataLoader,
    ColoredDataLoader,
    BlurredDataLoader,
    FilteredDataLoader,
    HeadPoseDataLoader,
    EthnicityDataLoader,
    CachedDataLoader,
)

from loreal_poc.models.wrappers import OpenCVWrapper, FaceAlignmentWrapper
from loreal_poc.tests.performance import NMEMean
from loreal_poc.tests.base import Test, TestDiff
from loreal_poc.marks.facial_parts import FacialParts

### Loading FFHQ dataset

In [2]:
dl_ref = DataLoaderFFHQ("ffhq")

### Loading landmark-detection models
- FaceAlignment
- OpenCV

In [3]:
models = {
    "FaceAlignment": FaceAlignmentWrapper(model=FaceAlignment(LandmarksType.TWO_D, device="cpu", flip_input=False)),
    "OpenCV": OpenCVWrapper(),
}
models.pop("FaceAlignment")  # takes a long time

loading data from : lbfmodel.yaml


In [4]:
results = []

### Case 1: Cropped Images

In [5]:
facial_parts = [FacialParts.LEFT_HALF.value, FacialParts.RIGHT_HALF.value]

for model in models.values():
    for fp in facial_parts:
        dl = CroppedDataLoader(dl_ref, part=fp)
        results.append(
            TestDiff(metric=NMEMean, threshold=1)
            .run(
                model=model,
                dataloader=dl,
                dataloader_ref=dl_ref,
                facial_part=fp,
            )
            .to_dict()
        )

OpenCV(4.8.1) /Users/runner/work/opencv-python/opencv-python/opencv/modules/core/src/matrix.cpp:1193: error: (-15:Bad number of channels) The total width is not divisible by the new number of channels in function 'reshape'

OpenCV(4.8.1) /Users/runner/work/opencv-python/opencv-python/opencv/modules/core/src/matrix.cpp:1193: error: (-15:Bad number of channels) The total width is not divisible by the new number of channels in function 'reshape'

OpenCV(4.8.1) /Users/runner/work/opencv-python/opencv-python/opencv/modules/core/src/matrix.cpp:1193: error: (-15:Bad number of channels) The total width is not divisible by the new number of channels in function 'reshape'

OpenCV(4.8.1) /Users/runner/work/opencv-python/opencv-python/opencv/modules/core/src/matrix.cpp:1193: error: (-15:Bad number of channels) The total width is not divisible by the new number of channels in function 'reshape'

OpenCV(4.8.1) /Users/runner/work/opencv-python/opencv-python/opencv/modules/core/src/matrix.cpp:1193: er

### Case 2A: Resized Images

In [6]:
for model in models.values():
    dl = ResizedDataLoader(dl_ref, scales=0.5)
    results.append(
        TestDiff(metric=NMEMean, threshold=1)
        .run(
            model=model,
            dataloader=dl,
            dataloader_ref=dl_ref,
        )
        .to_dict()
    )

### Case 2B: Recolored Images

In [7]:
for model in models.values():
    transformed_dl = ColoredDataLoader(dl)
    results.append(
        TestDiff(metric=NMEMean, threshold=1)
        .run(
            model=model,
            dataloader=transformed_dl,
            dataloader_ref=dl,
        )
        .to_dict()
    )

### Case 2C: Blurred Images

In [8]:
for model in models.values():
    dl = BlurredDataLoader(dl_ref)
    results.append(
        TestDiff(metric=NMEMean, threshold=1)
        .run(
            model=model,
            dataloader=dl,
            dataloader_ref=dl_ref,
        )
        .to_dict()
    )

### Case 3: Head Pose

In [9]:
cached_dl = CachedDataLoader(HeadPoseDataLoader(dl_ref), cache_size=None, cache_img=False, cache_marks=False)


def positive_roll(elt):
    return elt[2]["headPose"]["roll"] > 0


def negative_roll(elt):
    return elt[2]["headPose"]["roll"] < 0


head_poses = [positive_roll, negative_roll]

for model in models.values():
    for hp in head_poses:
        dl = FilteredDataLoader(cached_dl, hp)
        results.append(
            TestDiff(metric=NMEMean, threshold=1)
            .run(
                model=model,
                dataloader=dl,
                dataloader_ref=dl_ref,
            )
            .to_dict()
        )

### Case 4: Ethnicity

In [10]:
ethnicity_dl = EthnicityDataLoader(dl_ref, ethnicity_map={"indian": "asian"})
cached_dl = CachedDataLoader(ethnicity_dl, cache_size=None, cache_img=False, cache_marks=False)


def asian_ethnicity(elt):
    return elt[2]["ethnicity"] == "asian"


def white_ethnicity(elt):
    return elt[2]["ethnicity"] == "white"


ethnicities = [asian_ethnicity, white_ethnicity]

for model in models.values():
    for e in ethnicities:
        dl = FilteredDataLoader(cached_dl, e)
        results.append(
            TestDiff(metric=NMEMean, threshold=1)
            .run(
                model=model,
                dataloader=dl,
                dataloader_ref=dl_ref,
            )
            .to_dict()
        )

2024-01-09 10:50:51.969590: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [13]:
import pandas as pd

pd.set_option("display.max_colwidth", None)
report = pd.DataFrame(results)
# report.groupby(["model_name"]).apply(display) # doesn't work in CI

,test_name,metric_name,metric_value,threshold,passed,facial_part,model_name,dataloader_name
0,TestDiff,NME_mean,4.054881,1,False,left half,OpenCV,ffhq cropped on left half
1,TestDiff,NME_mean,11.737949,1,False,right half,OpenCV,ffhq cropped on right half
2,TestDiff,NME_mean,0.068788,1,True,entire face,OpenCV,ffhq resizing with ratios: 0.5
3,TestDiff,NME_mean,0.032044,1,True,entire face,OpenCV,ffhq resizing with ratios: 0.5 altered with color mode 7
4,TestDiff,NME_mean,0.065431,1,True,entire face,OpenCV,ffhq blurred
5,TestDiff,NME_mean,0.232795,1,True,entire face,OpenCV,(Cached (ffhq) with head-pose) filtered using 'positive_roll'
6,TestDiff,NME_mean,0.133026,1,True,entire face,OpenCV,(Cached (ffhq) with head-pose) filtered using 'negative_roll'
7,TestDiff,NME_mean,0.354641,1,True,entire face,OpenCV,(Cached (ffhq) with ethnicity) filtered using 'asian_ethnicity'
8,TestDiff,NME_mean,0.333034,1,True,entire face,OpenCV,(Cached (ffhq) with ethnicity) filtered using 'white_ethnicity'


""
